# Import der Libraries

In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: christoph-bieritz (cb-ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import os
import wandb
import numpy as np
import pandas as pd
import tensorflow as tf
import gymnasium as gym
import tensorflow_probability as tfp
import tensorflow.keras.losses as kls

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
from gymnasium import spaces
from google.colab import drive
from typing import List, Dict, Optional, Callable

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_path = '/content/drive/MyDrive/Data/PPO_portfolio_optimization/DS1_Winner_B&H.csv'
return_cols = pd.read_csv('/content/drive/MyDrive/Data/PPO_portfolio_optimization/DS1_Winner_BH.csv',index_col=0).columns
# --- Hyperparameter für das Training ---
num_iterations = 250  # Wie viele Zyklen aus Sammeln & Trainieren
steps_per_rollout = 2048  # Schritte pro Datensammlung (Batchgröße)
num_ppo_epochs = 10  # Wie oft über denselben Batch trainieren
mini_batch_size = 512  # Mini-Batch-Größe für das Training
gamma = 0.99  # Discount-Faktor
lambda_gae = 0.95  # GAE-Parameter
learning_rate=0.001

# --- PPO Hyperparameter ---
ppo_clip_epsilon = 0.2
vf_loss_coeff = 0.5
entropy_coeff = 0.01

initial_balance = 10000.0
window_size = 30
transaction_cost_pct = 0.000
sharpe_window = 20
sharpe_ma_window = 10

# --- Network Architecture ---
# TBD

# --- Optimierer ---
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# --- Loss-Funktion (HIER DIE NEUE ZEILE HINZUFÜGEN) ---
critic_loss_fn = tf.keras.losses.MeanSquaredError()


In [ ]:
def wandb_init():# Start a new wandb run to track this script.
  run = wandb.init(
      # Set the wandb entity where your project will be logged (generally your team name).
      entity="cb-ml", # Changed to your entity
      # Set the wandb project where this run will be logged.
      project="deep_learning_reinforcement_learning",# Make sure this project exists or change the name
      save_code=True,
      monitor_gym=True,
      reinit=True,
      # Track hyperparameters and run metadata.
      config={
          "num_iterations": num_iterations,
          "steps_per_rollout": steps_per_rollout,
          "num_ppo_epochs": num_ppo_epochs,
          "mini_batch_size": mini_batch_size,
          "gamma": gamma,
          "lambda_gae": lambda_gae,
          "learning_rate": learning_rate,
          "ppo_clip_epsilon": ppo_clip_epsilon,
          "vf_loss_coeff": vf_loss_coeff,
          "entropy_coeff": entropy_coeff,
          "initial_balance": initial_balance,
          "window_size": window_size,
          "transaction_cost_pct": transaction_cost_pct,
          "sharpe_window": sharpe_window,
          "sharpe_ma_window": sharpe_ma_window
      },
  )
  return run

run = wandb_init()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: christoph-bieritz (cb-ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Hier ist der Weights and Biases Code zum tracken der Konfigurationen der Experimente.

# Environment Class

In [ ]:
# (ZELLE 7 KORRIGIERT)
class PortfolioEnv(gym.Env):
    """
    Eine Gymnasium-Umgebung für das Portfolio-Management, die Daten aus einer
    CSV-Datei lädt und eine Trailing-Sharpe-Ratio als Belohnung verwendet.
    """

    metadata = {"render_modes": ["human"]}

    def __init__(
        self,
        csv_path: str,
        return_cols: List[str],
        initial_balance: float = initial_balance,
        window_size: int = window_size,
        transaction_cost_pct: float = transaction_cost_pct,
        sharpe_window: int = sharpe_window,
    ):

        super().__init__()

        # --- Parameter speichern ---
        self.csv_path = csv_path
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.transaction_cost_pct = transaction_cost_pct
        self.sharpe_window = sharpe_window

        # --- Asset-Konfiguration ---
        self.asset_return_columns = return_cols
        self.num_assets = len(self.asset_return_columns)
        self.num_portfolio_components = self.num_assets + 1

        # --- Daten laden ---
        self._load_data_csv() # Diese Funktion ist jetzt korrigiert

        # --- Interne Zustandsvariablen ---
        self.start_tick = self.window_size
        self.end_tick = len(self.df) - 1

        self.current_step = 0
        self.portfolio_value = 0.0
        self.prev_portfolio_value = 0.0
        self.portfolio_weights = np.zeros(
            self.num_portfolio_components, dtype=np.float32
        )
        self.done = False
        self._return_history = []
        self.sharpe_ma_window = sharpe_ma_window # Hyperparameter: Glättet die Sharpe über 10 Tage
        self._sharpe_history = []

        # --- 1. Action Space (Kontinuierlich) ---
        self.action_space = spaces.Box(
            low=0.0, high=1.0, shape=(self.num_portfolio_components,), dtype=np.float32
        )

        # --- 2. Observation Space (Dictionary) ---
        self.observation_space = spaces.Dict(
            {
                "market_data": spaces.Box(
                    low=-np.inf,
                    high=np.inf,
                    shape=(self.window_size, self.num_assets),
                    dtype=np.float32,
                ),
                "portfolio_weights": spaces.Box(
                    low=0.0,
                    high=1.0,
                    shape=(self.num_portfolio_components,),
                    dtype=np.float32,
                ),
            }
        )

    def _load_data_csv(self):
        try:
            # --- KORREKTUR HIER ---
            # self.csv_path (die Klassenvariable) verwenden, nicht die globale 'csv_path'
            self.df = pd.read_csv(self.csv_path)
            # --------------------

            if "Date" in self.df.columns:
                self.df["Date"] = pd.to_datetime(self.df["Date"])
                self.df.set_index("Date", inplace=True)

            if len(self.df) < self.window_size + 1:
                raise ValueError(
                    f"Nicht genügend Daten ({len(self.df)}) für window_size={self.window_size}."
                )
            print(
                f"Daten erfolgreich geladen: {len(self.df)} Zeitschritte, {self.num_assets} Assets."
            )
        except FileNotFoundError:
            print(f"Fehler: CSV-Datei {self.csv_path} nicht gefunden.")
            raise

    # (Rest der PortfolioEnv-Klasse bleibt unverändert)
    # ... (def _get_observation, def _calculate_reward, def reset, def step) ...
    # (Du musst den Rest der Klasse von deinem Original-Code kopieren, da er hier abgeschnitten ist)
    def _get_observation(self) -> Dict[str, np.ndarray]:
        end_idx = self.current_step
        start_idx = end_idx - self.window_size
        market_data = self.df[self.asset_return_columns].iloc[start_idx:end_idx].values

        obs_dict = {
            "market_data": market_data.astype(np.float32),
            "portfolio_weights": self.portfolio_weights.astype(np.float32),
        }
        return obs_dict

    def _calculate_reward(self) -> float:
        if self.prev_portfolio_value <= 1e-6:
            return 0.0
        reward = np.log(self.portfolio_value / self.prev_portfolio_value)
        if np.isnan(reward) or not np.isfinite(reward):
            reward = -10.0
        return reward

    def _calculate_reward_sharpe(self) -> float:
        if self.prev_portfolio_value > 1e-6:
            step_return = (self.portfolio_value / self.prev_portfolio_value) - 1.0
        else:
            step_return = 0.0
        self._return_history.append(step_return)
        current_sharpe = 0.0
        if len(self._return_history) >= self.sharpe_window:
            recent_returns = np.array(self._return_history[-self.sharpe_window:])
            mean_return = np.mean(recent_returns)
            std_dev = np.std(recent_returns)
            if std_dev > 1e-6:
                current_sharpe = (mean_return / std_dev) * np.sqrt(252)
        self._sharpe_history.append(current_sharpe)
        if len(self._sharpe_history) < self.sharpe_ma_window:
            return 0.0
        ma_sharpe = np.mean(self._sharpe_history[-self.sharpe_ma_window:])
        reward = np.clip(ma_sharpe, -5.0, 5.0)
        return reward

    def reset(
        self, seed: Optional[int] = None, options: Optional[dict] = None
    ) -> (dict, dict):
        super().reset(seed=seed)
        self.current_step = self.start_tick
        self.portfolio_value = self.initial_balance
        self.prev_portfolio_value = self.initial_balance
        self.portfolio_weights = np.zeros(
            self.num_portfolio_components, dtype=np.float32
        )
        self.portfolio_weights[0] = 1.0
        self.done = False
        self._return_history = []
        observation = self._get_observation()
        info = {}
        self._sharpe_history = []
        return observation, info

    def step(self, action: np.ndarray) -> (dict, float, bool, bool, dict):
        if self.done:
            return (
                self._get_observation(),
                0.0,
                True,
                False,
                {"info": "Episode already finished."},
            )
        self.prev_portfolio_value = self.portfolio_value
        old_actual_weights = self.portfolio_weights
        action = np.maximum(action, 0.0)
        action_sum = np.sum(action)
        if action_sum > 1e-6:
             target_weights = action / action_sum
        else:
             target_weights = np.zeros(self.num_portfolio_components, dtype=np.float32)
             target_weights[0] = 1.0
        turnover = np.sum(np.abs(target_weights[1:] - old_actual_weights[1:]))
        costs = turnover * self.prev_portfolio_value * self.transaction_cost_pct
        portfolio_value_after_costs = self.prev_portfolio_value - costs
        self.portfolio_weights = target_weights
        current_returns = (
            self.df[self.asset_return_columns].iloc[self.current_step].values
        )
        returns_vector = np.concatenate(([0.0], current_returns)).astype(np.float32)
        self.portfolio_value = portfolio_value_after_costs * np.sum(self.portfolio_weights * (1 + returns_vector))
        if self.portfolio_value <= 1e-6:
            self.portfolio_value = 0.0
            self.done = True
        reward = self._calculate_reward()
        self.current_step += 1
        if self.current_step > self.end_tick:
            self.done = True
        observation = self._get_observation()
        info = {"portfolio_value": self.portfolio_value, "transaction_costs": costs}
        terminated = self.done
        truncated = False
        return observation, reward, terminated, truncated, info

    def close(self):
        pass

# 1. Das Keras-Modell (Das "Gehirn" 🧠)

In [ ]:
# Wir importieren die Verteilungs-Tools von TFP
tfd = tfp.distributions

def create_actor_critic_model(env: gym.Env) -> (tf.keras.Model, Callable):
    """
    Erstellt ein Keras Actor-Critic-Modell (1 Stamm, 2 Köpfe),
    das dynamisch auf den 'spaces.Dict' Observation Space der Umgebung passt
    und die Parameter für eine Dirichlet-Verteilung ausgibt.

    Args:
        env (gym.Env): Eine Instanz deiner PortfolioEnv, die
                       env.observation_space["market_data"] und
                       env.observation_space["portfolio_weights"] bereitstellt.

    Returns:
        (tf.keras.Model): Das kompilierbare Keras-Modell.
        (Callable): Eine Funktion, die Alpha-Parameter entgegennimmt und
                    eine Dirichlet-Verteilung zurückgibt.
    """

    # --- 1. Definiere die Inputs (dynamisch aus der Env) ---
    market_input = Input(
        shape=env.observation_space["market_data"].shape,
        name="market_data",  # <-- Muss "market_data" heißen
    )

    weights_input = Input(
        shape=env.observation_space["portfolio_weights"].shape,
        name="portfolio_weights",  # <-- Muss "portfolio_weights" heißen
    )

    # --- 2. Gemeinsamer Stamm (Body) ---

    # Das LSTM verarbeitet die 2D-Marktdaten
    # (64 Neuronen ist ein Hyperparameter zum Tunen)
    lstm_out = LSTM(64, name="lstm_stamm")(market_input)

    # Kombiniere den LSTM-Output (Markt-Kontext) mit den
    # aktuellen Gewichten (aktueller Zustand)
    concatenated_features = Concatenate(name="kombinierte_features")(
        [lstm_out, weights_input]
    )

    # Ein gemeinsamer Dense-Layer für die kombinierten Features
    # (32 Neuronen ist ein Hyperparameter)
    shared_dense = Dense(32, activation="relu", name="gemeinsamer_dense")(
        concatenated_features
    )

    # --- 3. Definiere die 2 "Köpfe" ---

    # A) Der Actor-Kopf (gibt ALPHAS für die Dirichlet-Verteilung aus)
    actor_output_size = env.action_space.shape[0]  # (Assets + Cash)

    # Wir brauchen positive Alphas.
    # 'softplus' + 1.0 ist ein gängiger Trick, um alpha > 1 zu garantieren,
    # was die Verteilung stabilisiert (vermeidet extreme 0/1-Gewichte).
    alpha_params = Dense(
        actor_output_size,
        activation=lambda x: tf.nn.softplus(x) + 1.0,
        name="alpha_params",
    )(shared_dense)

    # B) Der Critic-Kopf (gibt den Wert des Zustands aus)
    # Nur ein Neuron, 'linear' (keine Aktivierung), da es einen Wert schätzt.
    critic_head = Dense(1, activation="linear", name="critic_output")(shared_dense)

    # --- 4. Erstelle das finale Modell ---
    # Das Modell weiß jetzt, dass es ein Dictionary mit den Schlüsseln
    # "market_data" und "portfolio_weights" als Input erwartet.
    model = Model(
        inputs=[market_input, weights_input],
        outputs=[alpha_params, critic_head],
        name="Actor_Critic_Dirichlet_Modell",
    )

    # --- 5. Hilfsfunktion für die Verteilung ---
    # Diese Funktion wird im Trainings-Loop verwendet, um aus den
    # Alpha-Parametern eine Verteilung zu erstellen.
    def create_distribution(alpha_params: tf.Tensor) -> tfd.Dirichlet:
        # Erstellt eine Dirichlet-Verteilung aus den Alpha-Parametern
        return tfd.Dirichlet(alpha_params)

    return model, create_distribution

# 2. Der Trainings-Schritt (Das "Nervensystem" mit GradientTape)

In [ ]:
def calculate_gae(rewards, values, next_values, dones, gamma=0.99, lambda_gae=0.95):
    """
    Berechnet die Generalized Advantage Estimation (GAE).
    """
    advantages = np.zeros_like(rewards)
    last_gae_lam = 0

    # (1.0 - dones) stellt sicher, dass der Wert 0 ist, wenn die Episode endet
    non_terminal = 1.0 - dones

    for t in reversed(range(len(rewards))):
        # Berechne den TD-Error (die "Überraschung")
        delta = rewards[t] + gamma * next_values[t] * non_terminal[t] - values[t]

        # Berechne den GAE-Advantage
        last_gae_lam = delta + gamma * lambda_gae * non_terminal[t] * last_gae_lam
        advantages[t] = last_gae_lam

    # 'targets' sind die Advantages + die Wertschätzungen
    targets = advantages + values
    return advantages, targets

@tf.function
def train_step(
    model, create_dist, observations_dict, actions, old_log_probs, advantages, targets
):
    """
    Führt den PPO-Trainingsschritt mit tf.GradientTape aus.
    """
    with tf.GradientTape() as tape:
        # --- 1. Forward Pass (Neue Werte berechnen) ---
        # Hole die *neuen* Alpha-Parameter und den *neuen* Wert
        new_alphas, new_critic_value = model(observations_dict, training=True)

        # Erstelle die *neue* Verteilung
        new_dist = create_dist(new_alphas)

        # Berechne die Log-Wahrscheinlichkeit der *alten* Aktionen unter der *neuen* Policy
        new_log_probs = new_dist.log_prob(actions)

        # --- 2. Critic-Verlust (Value Loss) ---
        # Rufe die Instanz auf, die wir oben erstellt haben
        critic_loss = critic_loss_fn(targets, new_critic_value)  # <-- KORRIGIERTE ZEILE

        # tf.reduce_mean ist nicht mehr nötig, da die Klasse das automatisch macht
        critic_loss = critic_loss * vf_loss_coeff

        # --- 3. Actor-Verlust (PPO-Loss) ---
        # Berechne das Wahrscheinlichkeits-Verhältnis (Ratio)
        ratio = tf.exp(new_log_probs - old_log_probs)

        # Clipped-Verlust
        clipped_ratio = tf.clip_by_value(
            ratio, 1 - ppo_clip_epsilon, 1 + ppo_clip_epsilon
        )

        actor_loss_unclipped = ratio * advantages
        actor_loss_clipped = clipped_ratio * advantages

        # Nimm das Minimum (schlechteste von beiden), um "gierige" Updates zu bestrafen
        actor_loss = -tf.reduce_mean(
            tf.minimum(actor_loss_unclipped, actor_loss_clipped)
        )

        # --- 4. Entropie-Verlust (für Exploration) ---
        # Wir wollen die Entropie MAXIMIEREN, also MINIMIEREN wir die negative Entropie
        entropy_loss = -tf.reduce_mean(new_dist.entropy())

        # --- 5. Gesamtverlust ---
        total_loss = actor_loss + critic_loss + (entropy_loss * entropy_coeff)

    # --- 6. Gradienten berechnen & anwenden ---
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss, actor_loss, critic_loss, entropy_loss

In [ ]:
# --- Datensatz-Pfade (global definieren) ---
BASE_DIR = "/content/drive/MyDrive/Data/PPO_portfolio_optimization/"
scenario_keys = [
    "DS1_Winner_BH", "DS2_Perfect_Timing", "DS3_Signal_in_Noise",
    "DS4_Risk_Choice", "DS5_Regime_Switch", "DS6_Correlation_Trap",
    "DS7_Needle_in_Haystack", "DS8_Sector_Diversification",
    "DS9_Factor_Timing", "DS10_SP100_Sim"
]
dataset_paths = [os.path.join(BASE_DIR, f"{key}.csv") for key in scenario_keys]

# --- W&B Login (global, einmalig) ---
# (Stelle sicher, dass wandb.login() in einer vorherigen Zelle aufgerufen wurde)


# --- 3. Starte den "Sweep" über alle Datensätze ---
for path in dataset_paths:

    # --- A. Initialisierung (WIRD JETZT IN JEDER SCHLEIFE AUSGEFÜHRT) ---
    print(f"\n--- Initialisiere Umgebung und Modell für {os.path.basename(path)} ---")

    CSV_FILE_PATH = path

    # --- KORREKTUR FÜR FEHLER 1 (KeyError) ---
    try:
        all_cols = pd.read_csv(CSV_FILE_PATH, nrows=0).columns.tolist()
        RETURN_COLUMNS = [col for col in all_cols if col != 'Date']
        print(f"Gefundene Return-Spalten: {RETURN_COLUMNS}")

        # Prüfe, ob überhaupt Spalten gefunden wurden (wichtig für DS8-Fehler)
        if not RETURN_COLUMNS:
            print(f"WARNUNG: Keine Return-Spalten in {path} gefunden. Überspringe.")
            continue # Gehe zur nächsten Datei
    except Exception as e:
        print(f"Konnte CSV nicht lesen oder Spalten nicht finden: {e}")
        continue
    # -------------------------------------------

    env = PortfolioEnv(
        csv_path=CSV_FILE_PATH,
        return_cols=RETURN_COLUMNS,
        window_size=window_size,
        sharpe_window=sharpe_window
    )

    model, create_distribution = create_actor_critic_model(env)

    # --- KORREKTUR FÜR FEHLER 2 (NotImplementedError) ---
    # Definiere Optimizer und Loss-Funktion HIER, INNENHALB der Schleife
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    critic_loss_fn = tf.keras.losses.MeanSquaredError()
    # --------------------------------------------------

    # --- KORREKTUR FÜR FEHLER 2 (NotImplementedError) ---
    # Definiere die kompilierte Funktion HIER, INNENHALB der Schleife
    @tf.function
    def train_step(model, create_dist, optimizer, observations_dict, actions, old_log_probs, advantages, targets):
        """
        Führt den PPO-Trainingsschritt mit tf.GradientTape aus.
        """
        with tf.GradientTape() as tape:
            new_alphas, new_critic_value = model(observations_dict, training=True)
            new_dist = create_dist(new_alphas)

            actions_clipped = tf.clip_by_value(actions, 1e-10, 1.0)
            new_log_probs = new_dist.log_prob(actions_clipped)

            critic_loss = critic_loss_fn(targets, new_critic_value)
            critic_loss = critic_loss * vf_loss_coeff

            ratio = tf.exp(new_log_probs - old_log_probs)
            clipped_ratio = tf.clip_by_value(ratio, 1 - ppo_clip_epsilon, 1 + ppo_clip_epsilon)
            actor_loss_unclipped = ratio * advantages
            actor_loss_clipped = clipped_ratio * advantages
            actor_loss = -tf.reduce_mean(tf.minimum(actor_loss_unclipped, actor_loss_clipped))

            entropy_loss = -tf.reduce_mean(new_dist.entropy())

            total_loss = actor_loss + critic_loss + (entropy_loss * entropy_coeff)

        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        return total_loss, actor_loss, critic_loss, entropy_loss
    # ----------------------------------------------------

    model.summary()

    # --- B. Starte W&B Run für dieses Szenario ---
    run = wandb.init(
        project="PPO_Portfolio_Sweep",
        name=f"{os.path.basename(path)} (lr={learning_rate})",
        config={
            "scenario": os.path.basename(path),
            "learning_rate": learning_rate,
            "window_size": window_size,
            "num_iterations": num_iterations,
            # (Füge hier alle anderen globalen Hyperparameter hinzu)
        },
        reinit=True
    )
    print("Starte Training...")

    # --- C. Haupt-Trainings-Loop (Unverändert) ---
    for iteration in range(num_iterations):

        # --- PHASE 1: DATENSAMMELN ---
        batch_obs, batch_actions, batch_rewards = [], [], []
        batch_old_log_probs, batch_values, batch_dones, batch_next_obs = [], [], [], []

        obs, info = env.reset()

        for t in range(steps_per_rollout):
            obs_market = np.expand_dims(obs["market_data"], axis=0)
            obs_weights = np.expand_dims(obs["portfolio_weights"], axis=0)
            model_inputs = {"market_data": obs_market, "portfolio_weights": obs_weights}

            alphas, critic_value = model(model_inputs)
            dist = create_distribution(alphas)
            action = dist.sample()[0].numpy()
            action = np.clip(action, 1e-10, 1.0)
            action = action / np.sum(action)
            old_log_prob = dist.log_prob(action).numpy()

            next_obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            batch_obs.append(obs); batch_actions.append(action); batch_rewards.append(reward)
            batch_old_log_probs.append(old_log_prob[0]); batch_values.append(critic_value.numpy()[0][0])
            batch_dones.append(done); batch_next_obs.append(next_obs)

            obs = next_obs
            if done:
                obs, info = env.reset()

        # --- PHASE 2: ADVANTAGES BERECHNEN ---
        obs_market_next = np.array([o["market_data"] for o in batch_next_obs])
        obs_weights_next = np.array([o["portfolio_weights"] for o in batch_next_obs])
        last_value_inputs = {
            "market_data": obs_market_next[-1:],
            "portfolio_weights": obs_weights_next[-1:]
        }

        # Behandele den Fall, dass der Batch leer ist (falls steps_per_rollout < 1)
        if not batch_values:
            print(f"WARNUNG: Kein einziger Schritt in Iteration {iteration} ausgeführt. Überspringe Training.")
            continue

        last_value = model(last_value_inputs)[1].numpy()[0][0]

        all_values = np.array(batch_values + [last_value], dtype=np.float32)
        batch_rewards_np = np.array(batch_rewards, dtype=np.float32)
        batch_dones_np = np.array(batch_dones, dtype=np.float32)

        rewards_mean = np.mean(batch_rewards_np); rewards_std = np.std(batch_rewards_np) + 1e-8
        normalized_rewards = (batch_rewards_np - rewards_mean) / rewards_std

        advantages, targets = calculate_gae(
            rewards=normalized_rewards, values=all_values[:-1], next_values=all_values[1:],
            dones=batch_dones_np, gamma=gamma, lambda_gae=lambda_gae
        )

        advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-8)

        # --- PHASE 3: TRAINIEREN ---
        obs_market_tensor = tf.convert_to_tensor([o["market_data"] for o in batch_obs])
        obs_weights_tensor = tf.convert_to_tensor([o["portfolio_weights"] for o in batch_obs])
        obs_dict_tensor = {"market_data": obs_market_tensor, "portfolio_weights": obs_weights_tensor}

        actions_tensor = tf.convert_to_tensor(batch_actions, dtype=tf.float32)
        old_log_probs_tensor = tf.convert_to_tensor(batch_old_log_probs, dtype=tf.float32)
        advantages_tensor = tf.convert_to_tensor(advantages, dtype=tf.float32)
        targets_tensor = tf.convert_to_tensor(targets, dtype=tf.float32)

        total_size = len(batch_obs)
        indices = np.arange(total_size)
        epoch_total_loss, epoch_actor_loss, epoch_critic_loss, epoch_entropy_loss = [], [], [], []

        for _ in range(num_ppo_epochs):
            np.random.shuffle(indices)
            for start in range(0, total_size, mini_batch_size):
                end = start + mini_batch_size
                mb_indices = indices[start:end]

                mb_obs_dict = {
                    "market_data": tf.gather(obs_market_tensor, mb_indices),
                    "portfolio_weights": tf.gather(obs_weights_tensor, mb_indices)
                }
                mb_actions = tf.gather(actions_tensor, mb_indices)
                mb_old_log_probs = tf.gather(old_log_probs_tensor, mb_indices)
                mb_advantages = tf.gather(advantages_tensor, mb_indices)
                mb_targets = tf.gather(targets_tensor, mb_indices)

                # Rufe train_step auf
                t_loss, a_loss, c_loss, e_loss = train_step(
                    model, create_distribution, optimizer,
                    mb_obs_dict, mb_actions, mb_old_log_probs, mb_advantages, mb_targets
                )

                epoch_total_loss.append(t_loss.numpy()); epoch_actor_loss.append(a_loss.numpy())
                epoch_critic_loss.append(c_loss.numpy()); epoch_entropy_loss.append(e_loss.numpy())

        # --- PHASE 4: Logging ---
        mean_reward = np.mean(batch_rewards)
        wandb.log({
            "Mean Reward": mean_reward,
            "Mean Total Loss": np.mean(epoch_total_loss),
            "Mean Actor Loss": np.mean(epoch_actor_loss),
            "Mean Critic Loss": np.mean(epoch_critic_loss),
            "Entropy": np.mean(epoch_entropy_loss)
        }, step=iteration)

        if iteration % 20 == 0:
            print(f"Iter {iteration}: Reward={mean_reward:.4f}, Critic Loss={np.mean(epoch_critic_loss):.4f}")

    wandb.finish()
    print(f"Training für {path} abgeschlossen.")


--- Initialisiere Umgebung und Modell für DS1_Winner_BH.csv ---
Gefundene Return-Spalten: ['Winner_Return', 'Loser_Return']
Daten erfolgreich geladen: 200 Zeitschritte, 2 Assets.


Model: "Actor_Critic_Dirichlet_Modell"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ market_data         │ (None, 30, 2)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_stamm (LSTM)   │ (None, 64)        │     17,152 │ market_data[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ portfolio_weights   │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ kombinierte_featur… │ (None, 67)        │          0 │ lstm_stamm[0][0], │
│ (Concatenate)       │                   │            │ portfolio_weight… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gemeinsamer_dense   │ (None, 32)        │      2,176 │ kombinierte_feat… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha_params        │ (None, 3)         │         99 │ gemeinsamer_dens… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ critic_output       │ (None, 1)         │         33 │ gemeinsamer_dens… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,460 (76.02 KB)

 Trainable params: 19,460 (76.02 KB)

 Non-trainable params: 0 (0.00 B)

Starte Training...
Iter 0: Reward=0.0000, Critic Loss=3.6880
Iter 20: Reward=0.0008, Critic Loss=4.1916
